In [1]:
import os
import gseapy as gp
import pandas as pd

In [2]:
os.listdir("C:/xuexi/Anaconda/RNA/used/rnk/")

['1. rpkm_rfe.rnk',
 '2. raw_counts_rfe.rnk',
 '3. raw_counts_rpkm_union_raw_counts_rfe_score.rnk',
 '4_raw_counts_rfe_8895_expressed_union_logFC.rnk',
 '6.raw_rpkm_intersection_raw_counts_rfe_score.rnk',
 '7. raw_rpkm_intersection_rpkm_rfe_score.rnk',
 '8. raw_counts_rpkm_union_raw_counts_logFC.rnk',
 'MM1S_CB_DMSO_8895_expressed_gene.rnk',
 'prompt.txt',
 'rf_rfe_rpkm_with_rfe_score.rnk',
 'rf_rfe_rpkm_with_rf_score.rnk']

In [4]:
extension = "symbols.gmt"
all_gene_sets = []
for filename in os.listdir(gene_sets_folder):
    if filename.endswith(extension):
        all_gene_sets.append(filename)

In [5]:
print(all_gene_sets)
print(len(all_gene_sets))

['c1.all.v2024.1.Hs.symbols.gmt', 'c2.cgp.v2024.1.Hs.symbols.gmt', 'c2.cp.biocarta.v2024.1.Hs.symbols.gmt', 'c2.cp.kegg_legacy.v2024.1.Hs.symbols.gmt', 'c2.cp.kegg_medicus.v2024.1.Hs.symbols.gmt', 'c2.cp.pid.v2024.1.Hs.symbols.gmt', 'c2.cp.reactome.v2024.1.Hs.symbols.gmt', 'c2.cp.wikipathways.v2024.1.Hs.symbols.gmt', 'c3.mir.mirdb.v2024.1.Hs.symbols.gmt', 'c3.mir.mir_legacy.v2024.1.Hs.symbols.gmt', 'c3.tft.gtrd.v2024.1.Hs.symbols.gmt', 'c3.tft.tft_legacy.v2024.1.Hs.symbols.gmt', 'c4.3ca.v2024.1.Hs.symbols.gmt', 'c4.cgn.v2024.1.Hs.symbols.gmt', 'c4.cm.v2024.1.Hs.symbols.gmt', 'c5.go.bp.v2024.1.Hs.symbols.gmt', 'c5.go.cc.v2024.1.Hs.symbols.gmt', 'c5.go.mf.v2024.1.Hs.symbols.gmt', 'c5.hpo.v2024.1.Hs.symbols.gmt', 'c6.all.v2024.1.Hs.symbols.gmt', 'c7.immunesigdb.v2024.1.Hs.symbols.gmt', 'c7.vax.v2024.1.Hs.symbols.gmt', 'c8.all.v2024.1.Hs.symbols.gmt', 'h.all.v2024.1.Hs.symbols.gmt']
24


In [ ]:
# all_gene_sets= ["h.all.v2024.1.Hs.symbols.gmt"]

In [ ]:
def run_gsea_for_geneset(gene_sets_):
    print
    try:
        # Perform GSEA
        gsea_results = gp.prerank(rnk=rank_path,
                                  gene_sets=gene_sets_,
                                  outdir=f"{output_path}/{rank_file.split('.')[0]}+{os.path.basename(gene_sets_)[0:15]}",
                                  min_size=5,
                                  max_size=500,
                                  permutation_num=1000,
#                                   weighted_score_type=1,
                                  ascending=False,
                                  seed = 42,
                                  threads=32)
        
        print(f"Completed GSEA for {gene_sets_}")
        return gsea_results
    except Exception as e:
        print(f"Error processing {gene_sets_}: {str(e)}")
        return None

In [ ]:
# Run GSEA for each gene set file
all_results = {}
all_valid_results = {}
for gene_set_file in all_gene_sets:
#     print(gene_set_file)
    results = run_gsea_for_geneset(gene_sets_folder+gene_set_file)
    if results.res2d is not None:
    
        results = pd.DataFrame(results.res2d)
        valid = results[(results['NES'].abs() >= 1.5) & (results['FDR q-val'] <= 0.05)]
        valid = valid.sort_values(by="NES")
        valid.to_csv(f"{output_path}/{rank_file.split('.')[0]}+{os.path.basename(gene_set_file)[0:15]}_VALID.csv",index=False)
    #     if results:
        all_results[os.path.basename(gene_set_file)] = results
    else:
        print("No results available for this gene set.")
    

In [6]:
rank_file_8895 = "MM1S_CB_DMSO_8895_expressed_gene"
rank_file_rfe_union = "3_raw_counts_rpkm_union_raw_counts_rfe_score"

In [7]:
_all_results = pd.DataFrame()
for gene_sets_file in all_gene_sets:
    rfe_results = pd.read_csv(output_path+rank_file_rfe_union+"+"+gene_sets_file[0:15]+"_VALID.csv")    
    if rfe_results.empty:
        rfe_results.loc[0] = None
    rfe_results = rfe_results.set_index("Term")
    rfe_results.columns = "3019gene_" + rfe_results.columns
    
        
    df_8895 = pd.read_csv(output_path+rank_file_8895+"+"+gene_sets_file[0:15]+"_VALID.csv")
    if df_8895.empty:
        df_8895.loc[0] = None
    df_8895 = df_8895.set_index("Term")
    df_8895.columns = "original_"+df_8895.columns
#     df_8895 = df_8895.reset_index()
    
    df_union = pd.concat([rfe_results,df_8895],axis=1)
    _all_results = pd.concat([_all_results,df_union], axis=0)
#     break

In [56]:
all_results = _all_results.dropna(how="all").reset_index()
all_results

,Term,3019gene_Name,3019gene_ES,3019gene_NES,3019gene_NOM p-val,3019gene_FDR q-val,3019gene_FWER p-val,3019gene_Tag %,3019gene_Gene %,3019gene_Lead_genes,original_Name,original_ES,original_NES,original_NOM p-val,original_FDR q-val,original_FWER p-val,original_Tag %,original_Gene %,original_Lead_genes
0,chr12q13,prerank,-0.448204,-2.345474,0.000000,0.013832,0.013,8/33,3.91%,INHBE;NDUFA4L2;GLI1;INHBC;NXPH4;ARHGAP9;SHMT2;...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chr16q22,prerank,0.490412,2.162725,0.003839,0.049680,0.052,12/19,25.97%,CMTM3;HAS3;SF3B3;HSF4;SLC7A6;TXNL4B;EXOSC6;PRM...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FISCHER_DREAM_TARGETS,prerank,-0.458210,-4.108158,0.000000,0.000000,0.000,237/277,43.16%,RDM1;RRM2;SHMT2;SLC1A5;ETV5;LONP1;GTPBP2;TSEN1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GOBERT_OLIGODENDROCYTE_DIFFERENTIATION_UP,prerank,-0.468281,-4.018646,0.000000,0.000000,0.000,186/211,43.92%,ANXA1;HSPB6;RDM1;RRM2;CREB5;ADAM23;GPC6;HIRIP3...,prerank,-0.641375,-2.923665,0.000000,0.000000,0.000,272/452,19.88%,ANXA1;RDM1;RRM2;ADAM23;MYB;DSCC1;TCF19;HMGN5;V...
4,BLANCO_MELO_BRONCHIAL_EPITHELIAL_CELLS_INFLUEN...,prerank,-0.557999,-3.993165,0.000000,0.000000,0.000,83/90,40.25%,PHGDH;RRM2;FADS1;EXO1;RAD54L;CDCA8;MCM2;ORC1;U...,prerank,-0.774161,-3.085713,0.000000,0.000000,0.000,102/128,14.60%,PHGDH;RRM2;FADS1;TCF19;CDC45;RAD51;CIT;WDR62;T...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,HALLMARK_BILE_ACID_METABOLISM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,prerank,-0.468241,-1.612635,0.012048,0.022647,0.207,15/52,15.55%,FADS1;IDH1;SLC27A2;PXMP2;PAOX;EPHX2;NEDD4;HACL...
1566,HALLMARK_GLYCOLYSIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,prerank,-0.393514,-1.571269,0.004392,0.027436,0.290,51/129,22.72%,IDH1;GFPT1;CTH;VEGFA;GOT1;HAX1;AURKA;RBCK1;GAL...
1567,HALLMARK_SPERMATOGENESIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,prerank,-0.464760,-1.560251,0.013865,0.027961,0.317,23/50,22.93%,GFI1;CLGN;NCAPH;RFC4;STRBP;CDKN3;PARP2;KIF2C;A...
1568,HALLMARK_INFLAMMATORY_RESPONSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,prerank,0.400259,1.573177,0.013624,0.045408,0.151,18/74,10.62%,CXCL10;RGS1;CD82;ICAM1;IL10RA;BTG2;TLR1;RTP4;T...


In [ ]:
# all_results.to_csv(output_path + "3019_genes.csv")

In [57]:
dff = pd.read_csv( "C:/xuexi/Anaconda/RNA/gene_sets_extracted.csv")
dff = dff.rename(columns={"STANDARD_NAME":"Term"})
dff["collection"] = dff["collection_name"].str.lower()
dff["collection"] = dff["collection"].str.replace(":",".")
dff = dff.drop("collection_name", axis=1)
dff = dff.sort_values(by="collection").reset_index(drop=True)
dff.tail(5)

,Term,PMID,EXACT_SOURCE,DESCRIPTION_BRIEF,DESCRIPTION_FULL,symbol,collection
34832,HALLMARK_INFLAMMATORY_RESPONSE,26771021.0,NaN,Genes defining inflammatory response.,NaN,"SLC7A2, STAB1, ATP2C1, KIF1B, DCBLD2, MXD1, GN...",h
34833,HALLMARK_IL6_JAK_STAT3_SIGNALING,26771021.0,NaN,Genes up-regulated by IL6 [GeneID=3569] via ST...,NaN,"TNFRSF12A, CD9, FAS, CD44, IL17RB, IL4R, IL12R...",h
34834,HALLMARK_IL2_STAT5_SIGNALING,26771021.0,NaN,Genes up-regulated by STAT5 in response to IL2...,NaN,"PHTF2, RNH1, RRAGD, CD44, PRKCH, IKZF2, CAPG, ...",h
34835,HALLMARK_OXIDATIVE_PHOSPHORYLATION,26771021.0,NaN,Genes encoding proteins involved in oxidative ...,NaN,"NDUFAB1, HCCS, SLC25A5, COX10, UQCRC1, MDH1, C...",h
34836,HALLMARK_UNFOLDED_PROTEIN_RESPONSE,26771021.0,NaN,Genes up-regulated during unfolded protein res...,NaN,"NFYA, EDC4, MTREX, HSPA5, HERPUD1, ALDH18A1, P...",h


In [14]:
dff["collection"].value_counts()

collection
c5.go.bp              7608
c5.hpo                5653
c7.immunesigdb        4872
c2.cgp                3494
c3.mir.mirdb          2377
c5.go.mf              1820
c2.cp.reactome        1736
c5.go.cc              1026
c8                     840
c2.cp.wikipathways     830
c2.cp.kegg_medicus     658
c3.tft.tft_legacy      610
c3.tft.gtrd            505
c4.cm                  431
c4.cgn                 427
c7.vax                 347
c1                     302
c2.cp.biocarta         292
c3.mir.mir_legacy      221
c2.cp.pid              196
c6                     189
c2.cp.kegg_legacy      186
c4.3ca                 148
h                       50
c2.cp                   19
Name: count, dtype: int64

In [11]:
len(dff)

34837

In [58]:
gene_set_number = pd.DataFrame(dff["collection"].value_counts()).sort_values(by="collection").reset_index()
gene_set_number.columns = ["collection","sets_count"]
gene_set_number

,collection,sets_count
0,c1,302
1,c2.cgp,3494
2,c2.cp,19
3,c2.cp.biocarta,292
4,c2.cp.kegg_legacy,186
5,c2.cp.kegg_medicus,658
6,c2.cp.pid,196
7,c2.cp.reactome,1736
8,c2.cp.wikipathways,830
9,c3.mir.mir_legacy,221


In [59]:
dff = dff.merge(gene_set_number,on="collection",how="left")
dff.head(2)

,Term,PMID,EXACT_SOURCE,DESCRIPTION_BRIEF,DESCRIPTION_FULL,symbol,collection,sets_count
0,chrYq12,NaN,Ensembl 112 human gene annotation on the GRCh3...,Ensembl 112 genes in cytogenetic band chrYq12,NaN,"VAMP7, DPH3P2, WASH6P, IL9R, WASIR1, TRPC6P1, ...",c1,302
1,chr2p24,NaN,Ensembl 112 human gene annotation on the GRCh3...,Ensembl 112 genes in cytogenetic band chr2p24,NaN,"PUM2, LAPTM4A, TRIB2, DDX1, SDC1, HS1BP3, LDAH...",c1,302


In [60]:
all_results = all_results.merge(dff,on = "Term", how="left")
all_results.head(2)

,Term,3019gene_Name,3019gene_ES,3019gene_NES,3019gene_NOM p-val,3019gene_FDR q-val,3019gene_FWER p-val,3019gene_Tag %,3019gene_Gene %,3019gene_Lead_genes,...,original_Tag %,original_Gene %,original_Lead_genes,PMID,EXACT_SOURCE,DESCRIPTION_BRIEF,DESCRIPTION_FULL,symbol,collection,sets_count
0,chr12q13,prerank,-0.448204,-2.345474,0.000000,0.013832,0.013,8/33,3.91%,INHBE;NDUFA4L2;GLI1;INHBC;NXPH4;ARHGAP9;SHMT2;...,...,NaN,NaN,NaN,NaN,Ensembl 112 human gene annotation on the GRCh3...,Ensembl 112 genes in cytogenetic band chr12q13,NaN,"CALCOCO1, HSD17B6, HOXC8, LIMA1, LETMD1, SLC4A...",c1,302
1,chr16q22,prerank,0.490412,2.162725,0.003839,0.049680,0.052,12/19,25.97%,CMTM3;HAS3;SF3B3;HSF4;SLC7A6;TXNL4B;EXOSC6;PRM...,...,NaN,NaN,NaN,NaN,Ensembl 112 human gene annotation on the GRCh3...,Ensembl 112 genes in cytogenetic band chr16q22,NaN,"EDC4, CDH1, RIPOR1, PHLPP2, CBFB, NFATC3, PDPR...",c1,302


In [64]:
def only_rfe(row):
    if pd.isna(row["original_NES"]):
        return True
    else:
        return False
    
def by_rfe(row):
    if not pd.isna(row["3019gene_NES"]):
        return True
    else:
        return False
    
def only_original(row):
    if pd.isna(row["3019gene_NES"]):
        return True
    else:
        return False
def by_original(row):
    if not pd.isna(row["original_NES"]):
        return True
    else:
        return False
def by_two(row):
    if (not pd.isna(row["original_NES"])) & (not pd.isna(row["3019gene_NES"])):
        return True
    else:
        return False
    
def imporve_by_rfe(row):
    if row["original_NES"]:
        if (abs(row["3019gene_NES"])>abs(row["original_NES"])) & (row["3019gene_FDR q-val"] <= row["original_FDR q-val"]):
            return True
        else:
            return False
    else:
        return False

In [65]:
all_results["only_rfe"] = all_results.apply(lambda row:only_rfe(row),axis=1)
all_results["by_rfe"] = all_results.apply(lambda row:by_rfe(row),axis=1)
all_results["only_original"] = all_results.apply(lambda row:only_original(row),axis=1)
all_results["only_rfe"] = all_results.apply(lambda row:only_rfe(row),axis=1)
all_results["by_rfe"] = all_results.apply(lambda row:by_rfe(row),axis=1)
all_results["by_original"] = all_results.apply(lambda row:by_original(row),axis=1)
all_results["by_two"] = all_results.apply(lambda row:by_two(row),axis=1)
all_results["improved_by_rfe"] = all_results.apply(lambda row:imporve_by_rfe(row),axis=1)

In [66]:
all_results["improved_by_rfe"].value_counts()

improved_by_rfe
False    1292
True      278
Name: count, dtype: int64

In [24]:
df = all_results.copy()

In [21]:
search_words = ["chromatin remodeling","chromatin accessibility","multiple myeloma",
               "lymphoma","hematology","hematological cancer","bone marrow", "ruvbl",
               "chromatin", "myeloma", "lymph", "hema", "marrow"]
search_words_lower = [word.lower() for word in search_words]  # Ensure case insensitivity
# def assignrow(text, word):
#     if pd.isna(text):  # Handle NaN values
#         return 0
#     return 1 if word.lower() in str(text).lower() else 0

In [22]:
import re

In [25]:
# Function to find words and extract context
def find_words_context(text, search_words):
    if pd.isna(text):  # Handle missing values
        return {word: 0 for word in search_words}, None

    text_clean = re.sub(r"\s+", " ", text.strip())  # Normalize spaces
    matches = {word: [] for word in search_words}

    for search_word in search_words:
#         pattern = rf"\b{re.escape(search_word)}"
        pattern = rf"{re.escape(search_word)}" 
        if re.search(pattern, text_clean, re.IGNORECASE):  # Case-insensitive regex search
            match = re.search(pattern, text_clean, re.IGNORECASE)
            start, end = match.span()
            words = text_clean.split()
            i = words.index(search_word.split()[0]) if search_word.split()[0] in words else -1
            prev_word = words[i-1] if i > 0 else ""
            next_word = words[i+1] if i < len(words)-1 else ""
            matches[search_word].append(f"{prev_word} {search_word} {next_word}".strip())

    results = {word: (1 if matches[word] else 0) for word in search_words}
    matched_texts = "; ".join(sum(matches.values(), [])) if any(matches.values()) else None
    return results, matched_texts

# Apply function to both columns and combine results
df_results = df.apply(lambda row: pd.Series(find_words_context(row["DESCRIPTION_BRIEF"], search_words_lower)), axis=1)
df_results_full = df.apply(lambda row: pd.Series(find_words_context(row["DESCRIPTION_FULL"], search_words_lower)), axis=1)

# Extract match indicators and text
df_matches = df_results.iloc[:, 0].apply(pd.Series) | df_results_full.iloc[:, 0].apply(pd.Series)  # Logical OR
df["text"] = df_results.iloc[:, 1].fillna("") + "; " + df_results_full.iloc[:, 1].fillna("")
df["text"] = df["text"].str.strip().replace("^; ", "", regex=True).replace("; $", "", regex=True)
df["text"] = df["text"].replace("", None)  # If empty, set to None


# df.columns = df.columns.str.strip().str.lower()
# Add match indicator columns
for i, word in enumerate(search_words):
    df[word] = df_matches[word]

# Set 'related' column: 1 if any match is found, otherwise 0
# df["related"] = df_matches.max(axis=1)

In [ ]:
df["related"] = df_matches.max(axis=1)

In [ ]:
df.to_csv("./68immune.csv")

In [26]:
df["related_word8"] = df_matches.iloc[:,0:8].max(axis=1)

In [27]:
df["related_word6"] = df_matches.iloc[:,-6:].max(axis=1)

In [28]:
df.groupby(["collection"]).related_word6.sum().reset_index()

,collection,related_word6
0,c1,0
1,c2.cgp,103
2,c2.cp.biocarta,1
3,c2.cp.kegg_legacy,0
4,c2.cp.kegg_medicus,0
5,c2.cp.pid,0
6,c2.cp.reactome,1
7,c2.cp.wikipathways,0
8,c3.mir.mir_legacy,0
9,c3.mir.mirdb,0


In [29]:
df.groupby(["collection"]).related_word8.sum().reset_index()

,collection,related_word8
0,c1,0
1,c2.cgp,46
2,c2.cp.biocarta,0
3,c2.cp.kegg_legacy,0
4,c2.cp.kegg_medicus,0
5,c2.cp.pid,0
6,c2.cp.reactome,0
7,c2.cp.wikipathways,0
8,c3.mir.mir_legacy,0
9,c3.mir.mirdb,0


In [ ]:
df.groupby(["collection"]).related.sum().reset_index()

In [ ]:


# # Function to find words and extract context
# def find_words_context(text, search_words):
#     if pd.isna(text):  # Handle missing values
#         return (0, 0, 0, 0,0,0,0,0,0,0,0,0,0,None)

#     words = str(text).split()  # Convert text to lowercase for case-insensitive matching
#     words_lower = [word.lower() for word in words]
#     matches = {word: [] for word in search_words}

#     for i, word in enumerate(words_lower):
#         for search_word in search_words:
#             if word == search_word:
#                 prev_word = words[i-1] if i > 0 else ""
#                 next_word = words[i+1] if i < len(words)-1 else ""
#                 matches[search_word].append(f"{prev_word} {words[i]} {next_word}".strip())

#     results = {word: (1 if matches[word] else 0) for word in search_words}
#     matched_texts = "; ".join(sum(matches.values(), [])) if any(matches.values()) else 0
#     return (*results.values(), matched_texts)

# # Apply function to both columns and combine results
# df[["chromatin remodeling","chromatin accessibility","multiple myeloma",
#                "lymphoma","hematology","hematological cancer","bone marrow", "RUVBL",
#                "chromatin", "myeloma", "lymph", "hema", "marrow","Context"]] = df["DESCRIPTION_BRIEF"].astype(str).apply(
#     lambda x: pd.Series(find_words_context(x, search_words_lower))
# )

# df_temp = df["DESCRIPTION_FULL"].astype(str).apply(
#     lambda x: pd.Series(find_words_context(x, search_words_lower))
# )
# df.iloc[:,-14:-1] = df.iloc[:,-14:-1].fillna(0).astype(int)
# df_temp.iloc[:,0:13]  = df_temp.iloc[:,0:13].fillna(0).astype(int)
# for i, word in enumerate(search_words):
#     df[word] = df[word] | df_temp[i]
# # **Merge matched text results**
# df["Context"] = df["Context"].astype(str).fillna("") + "; " + df_temp[13].astype(str).fillna("")
# df["Context"] = df["Context"].str.strip().replace("^; ", "", regex=True).replace("; $", "", regex=True)
# df["Context"].replace("", None, inplace=True)  # If empty, set to None

# def isrelated_to_word8(row):
#     if row[-14:-1].sum() == 0:
#         return 0
#     else:
#         return 1
    
# df["is_related"] = df.apply(lambda row:isrelated_to_word8(row),axis=1 )

In [31]:
# ["only_rfe","by_rfe","only_original","by_original","by_two"]

In [ ]:
# df[df["only_rfe"]==True]

In [ ]:
# df["collection"].value_counts()

In [ ]:
# df[df["collection"]=="c7.immunesigdb"]

In [33]:
df_sub = df[["collection","Term","sets_count","only_rfe","by_rfe","only_original","by_original","by_two","text","related_word8","related_word6"]]
df_sub

,collection,Term,sets_count,only_rfe,by_rfe,only_original,by_original,by_two,text,related_word8,related_word6
0,c1,chr12q13,302,True,True,False,False,False,;,0,0
1,c1,chr16q22,302,True,True,False,False,False,;,0,0
2,c2.cgp,FISCHER_DREAM_TARGETS,3494,True,True,False,False,False,;,0,0
3,c2.cgp,GOBERT_OLIGODENDROCYTE_DIFFERENTIATION_UP,3494,False,True,False,True,True,;,0,0
4,c2.cgp,BLANCO_MELO_BRONCHIAL_EPITHELIAL_CELLS_INFLUEN...,3494,False,True,False,True,True,;,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1565,h,HALLMARK_BILE_ACID_METABOLISM,50,False,False,True,True,False,;,0,0
1566,h,HALLMARK_GLYCOLYSIS,50,False,False,True,True,False,;,0,0
1567,h,HALLMARK_SPERMATOGENESIS,50,False,False,True,True,False,;,0,0
1568,h,HALLMARK_INFLAMMATORY_RESPONSE,50,False,False,True,True,False,;,0,0


In [34]:
cols_to_check = ["only_rfe", "by_rfe", "only_original", "by_original", "by_two"]

In [ ]:
# Compute counts where "is_related" is 1 and column value is TRUE
result = df_sub[df_sub["related_word8"] == 1].groupby("collection")[cols_to_check].sum()

In [39]:
# Compute counts where "is_related" is 1 and column value is TRUE
related_counts = df_sub[df_sub["related_word8"] == 1].groupby("collection")[cols_to_check].sum()

# Compute total count of TRUE values for each checked column per collection
total_counts = df_sub.groupby("collection")[cols_to_check].sum()

# Merging both results
final_result_word8 = related_counts.join(total_counts, lsuffix="_related", rsuffix="_total")
final_result_word8

,only_rfe_related,by_rfe_related,only_original_related,by_original_related,by_two_related,only_rfe_total,by_rfe_total,only_original_total,by_original_total,by_two_total
collection,,,,,,,,,,
c2.cgp,7,36,10,39,29,63,256,130,323,193
c4.cm,0,0,1,1,0,2,25,32,55,23
c6,0,1,0,1,1,3,24,11,32,21
c7.immunesigdb,5,28,41,64,23,34,193,276,435,159
c7.vax,1,1,0,0,0,15,33,15,33,18
c8,11,17,17,23,6,23,73,71,121,50


In [40]:
# Compute counts where "is_related" is 1 and column value is TRUE
related_counts = df_sub[df_sub["related_word6"] == 1].groupby("collection")[cols_to_check].sum()

# Compute total count of TRUE values for each checked column per collection
total_counts = df_sub.groupby("collection")[cols_to_check].sum()

# Merging both results
final_result_word6 = related_counts.join(total_counts, lsuffix="_related", rsuffix="_total")
final_result_word6

,only_rfe_related,by_rfe_related,only_original_related,by_original_related,by_two_related,only_rfe_total,by_rfe_total,only_original_total,by_original_total,by_two_total
collection,,,,,,,,,,
c2.cgp,21,77,26,82,56,63,256,130,323,193
c2.cp.biocarta,0,0,1,1,0,0,0,2,2,0
c2.cp.reactome,0,1,0,1,1,5,47,43,85,42
c4.cm,0,0,1,1,0,2,25,32,55,23
c5.go.bp,0,0,3,3,0,13,43,55,85,30
c5.go.cc,0,0,1,1,0,2,5,12,15,3
c6,0,2,0,2,2,3,24,11,32,21
c7.immunesigdb,10,51,90,131,41,34,193,276,435,159
c7.vax,5,13,4,12,8,15,33,15,33,18


In [38]:
final_result

,only_rfe_related,by_rfe_related,only_original_related,by_original_related,by_two_related,only_rfe_total,by_rfe_total,only_original_total,by_original_total,by_two_total
collection,,,,,,,,,,
c2.cgp,21,77,26,82,56,63,256,130,323,193
c2.cp.biocarta,0,0,1,1,0,0,0,2,2,0
c2.cp.reactome,0,1,0,1,1,5,47,43,85,42
c4.cm,0,0,1,1,0,2,25,32,55,23
c5.go.bp,0,0,3,3,0,13,43,55,85,30
c5.go.cc,0,0,1,1,0,2,5,12,15,3
c6,0,2,0,2,2,3,24,11,32,21
c7.immunesigdb,10,51,90,131,41,34,193,276,435,159
c7.vax,5,13,4,12,8,15,33,15,33,18


In [41]:
df.to_csv("C:/xuexi/Anaconda/RNA/msigdb/3019genes_raw_counts_rpkm_rfe_union_final.csv")

In [42]:
final_result_word8.to_csv("./msigdb/3019genes_related_word8.csv")
final_result_word6.to_csv("./msigdb/3019genes_related_word6.csv")